In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.io as pio
import datetime
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


# Enable Dash in Jupyter
pio.renderers.default = "iframe"

# Google Sheets API Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"

# Authenticate and connect to Google Sheets
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)

# Load data into a Pandas DataFrame
data = pd.DataFrame(worksheet.get_all_records())

# Ensure each athlete appears only once and sort alphabetically
unique_athletes = data.groupby("ID").first().reset_index()
unique_athletes = unique_athletes.sort_values(by=["First Name", "Last Name"])  # 🔥 Sort alphabetically

# Create dropdown options
athlete_options = [
    {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
    for _, row in unique_athletes.iterrows()
]

athlete_options = [
    {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
    for _, row in unique_athletes.iterrows()
]

# Initialize the Dash app
app = Dash(__name__)

# Modify the layout to include Trainer and Status dropdowns
app.layout = html.Div([
    html.H1("BEA Athlete Performance Dashboard"),

    # Trainer Dropdown
    html.Label("Select a Trainer:"),
    dcc.Dropdown(
        id="trainer-dropdown",
        options=[{"label": trainer, "value": trainer} for trainer in sorted(data["Trainer"].dropna().unique())],
        placeholder="Choose a trainer (Optional)",
        style={"width": "50%"},
        clearable=True,
        persistence=True,  # 🔥 Retains selection across refreshes
        persistence_type="session"  # 🔥 Saves selection per browser session
    ),

    # Status Dropdown
    html.Label("Select Athlete Status:"),
    dcc.Dropdown(
        id="status-dropdown",
        options=[{"label": status, "value": status} for status in sorted(data["Status"].dropna().unique())],
        placeholder="Choose a status (Optional)",
        style={"width": "50%"},
        clearable=True,
        persistence=True,
        persistence_type="session"
    ),

    # Athlete Dropdown (Filtered by Trainer & Status)
    html.Label("Select an Athlete:"),
    dcc.Dropdown(
        id="athlete-dropdown",
        placeholder="Choose an athlete...",
        style={"width": "50%"},
        persistence=True,
        persistence_type="session"
    ),

    html.Div(id="athlete-info"),  # Athlete details will always be displayed

    dcc.Tabs(id="tabs", value="velocity", children=[
        dcc.Tab(label="Velocity Trends", value="velocity"),
        dcc.Tab(label="Strength Trends", value="strength"),
        dcc.Tab(label="Test Comparisons", value="test-comparisons"),
    ]),

    html.Div(id="tabs-content"),  # Selected tab content

    # 🔥 Auto-refresh data every 5 minutes
    dcc.Interval(
        id="interval-component",
        interval=5*60*1000,  # Refresh every 5 minutes
        n_intervals=0
    )
])



from dash.dependencies import Input, Output

# Callback to filter athlete dropdown based on selected trainer
@app.callback(
    Output("athlete-dropdown", "options"),
    [Input("trainer-dropdown", "value"),
     Input("status-dropdown", "value"),
     Input("interval-component", "n_intervals")]  # 🔥 Triggers refresh every few minutes
)
def filter_athletes(selected_trainer, selected_status, n_intervals):
    # 🔥 Refresh data from Google Sheets
    worksheet = sheet.get_worksheet(0)
    global data
    data = pd.DataFrame(worksheet.get_all_records())

    filtered_athletes = data.copy()
    
    if selected_trainer:
        filtered_athletes = filtered_athletes[filtered_athletes["Trainer"] == selected_trainer]

    if selected_status:
        filtered_athletes = filtered_athletes[filtered_athletes["Status"] == selected_status]

    filtered_athletes = filtered_athletes.groupby("ID").first().reset_index()
    filtered_athletes = filtered_athletes.sort_values(by=["First Name", "Last Name"])

    return [
        {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
        for _, row in filtered_athletes.iterrows()
    ]


# Callback to update athlete info and tab content
@app.callback(
    [Output("athlete-info", "children"), Output("tabs-content", "children")],
    [Input("athlete-dropdown", "value"), Input("tabs", "value")]
)
def update_dashboard(selected_id, selected_tab):
    if selected_id is None:
        return "Select an athlete to view details.", "Select a tab to view data."

    # Get athlete details
    athlete_tests = data[data["ID"] == selected_id].copy()
    athlete_tests["Date"] = pd.to_datetime(athlete_tests["Date"], errors="coerce")
    athlete_tests_sorted = athlete_tests.sort_values(by="Date", ascending=False)

    most_recent_level = athlete_tests_sorted["Level"].dropna().iloc[0] if not athlete_tests_sorted["Level"].dropna().empty else "Unknown"
    most_recent_school = athlete_tests_sorted["School/Org"].dropna().iloc[0] if not athlete_tests_sorted["School/Org"].dropna().empty else "Unknown"

    last_test_dates = {}
    today = pd.Timestamp.today()
    for workout in ["Strength", "Pitching", "Hitting"]:
        workout_tests = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == workout]
        if not workout_tests.empty:
            last_test_date = workout_tests.iloc[0]["Date"]
            days_since_test = (today - last_test_date).days
            last_test_dates[workout] = f"{last_test_date.strftime('%Y-%m-%d')} 🚨 Due for Testing" if days_since_test > 60 else last_test_date.strftime('%Y-%m-%d')
        else:
            last_test_dates[workout] = "No Test Recorded"

    athlete_info = athlete_tests_sorted.iloc[0]
    
    athlete_info_display = html.Div([
        html.H3(f"{athlete_info['First Name']} {athlete_info['Last Name']}"),
        html.P(f"ID: {selected_id}"),
        html.P(f"Status: {athlete_info['Status']}"),
        html.P(f"Trainer: {athlete_info['Trainer']}"),
        html.P(f"Position: {athlete_info['Position']}"),
        html.P(f"Level: {most_recent_level}"),
        html.P(f"School/Org: {most_recent_school}"),
        html.H4("Last Test Dates:"),
        html.P(f"Strength: {last_test_dates['Strength']}"),
        html.P(f"Pitching: {last_test_dates['Pitching']}"),
        html.P(f"Hitting: {last_test_dates['Hitting']}"),
    ])

    # Handle Tabs (Velocity Trends, Strength Trends, Test Comparisons)
    if selected_tab == "velocity":
        pitching_data = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == "Pitching"]

        if pitching_data.empty:
            tab_content = html.Div([html.H3("No velocity data available for this athlete.")])
        else:
            test_types = pitching_data["Test Type"].unique()
            graphs = []

            for test_type in test_types:
                test_data = pitching_data[pitching_data["Test Type"] == test_type]
                if test_data.empty:
                    continue

                y_axis = "Arm Score" if test_type == "ArmCare" else "Max Velo"
                y_label = "Arm Score" if test_type == "ArmCare" else "Max Velocity (MPH)"

            # 🔥 Find PRs for each Test Sub-Type
                pr_data = test_data.groupby("Test Sub-Type")[y_axis].transform(max)
                test_data["PR"] = test_data[y_axis] == pr_data  # Mark PR points for each Test Sub-Type

                fig = px.line(
                    test_data, x="Date", y=y_axis, color="Test Sub-Type", markers=True,
                    title=f"{test_type} - {y_label} Trends"
                )

            # 🔥 Add PR Annotations for each Test Sub-Type
                for _, row in test_data[test_data["PR"]].iterrows():
                    fig.add_annotation(
                        x=row["Date"], y=row[y_axis],
                        text=f"🔥 PR ({row['Test Sub-Type']})!", showarrow=True, arrowhead=2,
                        font=dict(color="red", size=12)
                    )

                fig.update_layout(xaxis_title="Date", yaxis_title=y_label)
                graphs.append(dcc.Graph(figure=fig))

            tab_content = html.Div(graphs)


    elif selected_tab == "strength":
        strength_data = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == "Strength"]

        if strength_data.empty:
            tab_content = html.Div([html.H3("No strength data available for this athlete.")])
        else:
            strength_tests = ["Back Squat", "Bench Press", "Deadlift"]
            graphs = []

            for test_type in strength_tests:
                test_data = strength_data[strength_data["Test Type"] == test_type]
                if test_data.empty:
                    continue

            # 🔥 Ensure date is formatted correctly (YYYY-MM-DD only)
                test_data["Date"] = test_data["Date"].dt.strftime('%Y-%m-%d')

            # 🔥 PR Detection: Find the highest Weight for each Test Sub-Type
                pr_weights = test_data.groupby("Test Sub-Type")["Weight"].max().reset_index()

            # 🔥 Within those, find the highest Speed for tiebreaker
                pr_data = pd.merge(test_data, pr_weights, on=["Test Sub-Type", "Weight"], how="inner")
                pr_data = pr_data.loc[pr_data.groupby("Test Sub-Type")["Speed"].idxmax()]

            # 🔥 Mark PR in the original dataset
                test_data["PR"] = test_data.apply(
                    lambda row: row["Weight"] == pr_data.loc[pr_data["Test Sub-Type"] == row["Test Sub-Type"], "Weight"].values[0]
                                and row["Speed"] == pr_data.loc[pr_data["Test Sub-Type"] == row["Test Sub-Type"], "Speed"].values[0]
                                if row["Test Sub-Type"] in pr_data["Test Sub-Type"].values else False,
                    axis=1
                )

            # 🔥 Create scatter plot (Weight vs. Speed)
                fig = px.scatter(
                    test_data, 
                    x="Speed", 
                    y="Weight", 
                    color="Test Sub-Type", 
                    title=f"{test_type} - Strength Trends",
                    hover_data={"Date": True, "Phase": True, "Weight": True, "Speed": True}  # Display extra info on hover
                )

            # 🔥 Add PR Annotations for each Test Sub-Type
                for _, row in test_data[test_data["PR"]].iterrows():
                    fig.add_annotation(
                        x=row["Speed"], y=row["Weight"],
                        text=f"🔥 PR ({row['Test Sub-Type']})!", showarrow=True, arrowhead=2,
                        font=dict(color="red", size=12)
                    )

                fig.update_layout(
                    xaxis_title="Speed (m/s)", 
                    yaxis_title="Weight (lbs)"
                )
                graphs.append(dcc.Graph(figure=fig))

            tab_content = html.Div(graphs)

    elif selected_tab == "test-comparisons":
        # Test Comparison Graphs
        comparison_data = athlete_tests_sorted[["Date", "Workout Type", "Max Velo", "Max Spin", "IVB", "Max Bat Speed", "Weight", "Speed"]].dropna(how="all")
        if comparison_data.empty:
            tab_content = html.Div([html.H3("No test comparison data available for this athlete.")])
        else:
            graphs = []
            pitching_data = comparison_data[comparison_data["Workout Type"] == "Pitching"]
            if not pitching_data.empty:
                fig_velo_spin = px.scatter(pitching_data, x="Max Velo", y="Max Spin", color="Workout Type",
                                           title="Max Velocity vs. Max Spin Rate")
                fig_velo_spin.update_layout(xaxis_title="Max Velocity (MPH)", yaxis_title="Max Spin Rate (RPM)")
                graphs.append(dcc.Graph(figure=fig_velo_spin))
            tab_content = html.Div(graphs)

    return athlete_info_display, tab_content

# Run the app in Jupyter Notebook
app.run_server(mode="inline", debug=True)


C:\Users\benoi\AppData\Local\Temp\ipykernel_10936\3989416899.py:200: FutureWarning:

The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.

C:\Users\benoi\AppData\Local\Temp\ipykernel_10936\3989416899.py:201: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\benoi\AppData\Local\Temp\ipykernel_10936\3989416899.py:200: FutureWarning:

The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.

C:\Users\benoi\AppData\Local\Temp\ipy

Okay lets move on to Test Comparison tab. This tab must be handled delicately because what I want this tab to do is highly complex.

As it is now there are pre programmed comparisons written in the script. Instead of this I would like this tab to act "dynamically" being able to select from an array of different options, as well as having background aggregate calculations such as average/max/min of the facility being done to accomplish things, given an empty plot such as...

1.) Select Workouts to create custom scatter plots that can be used to find correlations between Strength Workouts and Pitching or Hitting Tests...
The main challenge for this plot is date matching, because Strength, Pitching and Hitting tests do not occur simultaneously, there has to be a clever way to date match, whether that is by using the closest dates or possibly better yet, matching phases...

2.) Player to Player Comparisons

3.) Facility Trends by Phase, comparing the average results of the facility to the selected athletes growth in that phase.


Okay lets move on to Test Comparison tab. This tab must be handled delicately because what I want this tab to do is highly complex.

As it is now there are pre programmed comparisons written in the script. Instead of this I would like this tab to act "dynamically" being able to select from an array of different options, as well as having background aggregate calculations such as average/max/min of the facility being done to accomplish things, given an empty plot such as...

1.) Select Workouts to create custom scatter plots that can be used to find correlations between Strength Workouts and Pitching or Hitting Tests...
The main challenge for this plot is date matching, because Strength, Pitching and Hitting tests do not occur simultaneously, there has to be a clever way to date match, whether that is by using the closest dates or possibly better yet, matching phases...

2.) Player to Player Comparisons

3.) Facility Trends by Phase, comparing the average results of the facility to the selected athletes growth in that phase.

Please consult if any of these are possible, how I can elaborate on some ideas, as well as other ideas and plot types that would be beneficial to compare and contrast an athlete's progression with this data set


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.io as pio
import datetime
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


# Enable Dash in Jupyter
pio.renderers.default = "iframe"

# Google Sheets API Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"

# Authenticate and connect to Google Sheets
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)

# Load data into a Pandas DataFrame
data = pd.DataFrame(worksheet.get_all_records())

# Ensure each athlete appears only once and sort alphabetically
unique_athletes = data.groupby("ID").first().reset_index()
unique_athletes = unique_athletes.sort_values(by=["First Name", "Last Name"])  # 🔥 Sort alphabetically

# Create dropdown options
athlete_options = [
    {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
    for _, row in unique_athletes.iterrows()
]

athlete_options = [
    {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
    for _, row in unique_athletes.iterrows()
]

# Initialize the Dash app
app = Dash(__name__)

# Modify the layout to include Trainer and Status dropdowns
app.layout = html.Div([
    html.H1("BEA Athlete Performance Dashboard"),

    # Trainer Dropdown
    html.Label("Select a Trainer:"),
    dcc.Dropdown(
        id="trainer-dropdown",
        options=[{"label": trainer, "value": trainer} for trainer in sorted(data["Trainer"].dropna().unique())],
        placeholder="Choose a trainer (Optional)",
        style={"width": "50%"},
        clearable=True,
        persistence=True,  # 🔥 Retains selection across refreshes
        persistence_type="session"  # 🔥 Saves selection per browser session
    ),

    # Status Dropdown
    html.Label("Select Athlete Status:"),
    dcc.Dropdown(
        id="status-dropdown",
        options=[{"label": status, "value": status} for status in sorted(data["Status"].dropna().unique())],
        placeholder="Choose a status (Optional)",
        style={"width": "50%"},
        clearable=True,
        persistence=True,
        persistence_type="session"
    ),

    # Athlete Dropdown (Filtered by Trainer & Status)
    html.Label("Select an Athlete:"),
    dcc.Dropdown(
        id="athlete-dropdown",
        placeholder="Choose an athlete...",
        style={"width": "50%"},
        persistence=True,
        persistence_type="session"
    ),

    html.Div(id="athlete-info"),  # Athlete details will always be displayed

    dcc.Tabs(id="tabs", value="velocity", children=[
        dcc.Tab(label="Velocity Trends", value="velocity"),
        dcc.Tab(label="Strength Trends", value="strength"),
        dcc.Tab(label="Test Comparisons", value="test-comparisons"),
    ]),

    html.Div(id="tabs-content"),  # Selected tab content

    # 🔥 Auto-refresh data every 5 minutes
    dcc.Interval(
        id="interval-component",
        interval=5*60*1000,  # Refresh every 5 minutes
        n_intervals=0
    )
])



from dash.dependencies import Input, Output

# Callback to filter athlete dropdown based on selected trainer
@app.callback(
    Output("athlete-dropdown", "options"),
    [Input("trainer-dropdown", "value"),
     Input("status-dropdown", "value"),
     Input("interval-component", "n_intervals")]  # 🔥 Triggers refresh every few minutes
)
def filter_athletes(selected_trainer, selected_status, n_intervals):
    # 🔥 Refresh data from Google Sheets
    worksheet = sheet.get_worksheet(0)
    global data
    data = pd.DataFrame(worksheet.get_all_records())

    filtered_athletes = data.copy()
    
    if selected_trainer:
        filtered_athletes = filtered_athletes[filtered_athletes["Trainer"] == selected_trainer]

    if selected_status:
        filtered_athletes = filtered_athletes[filtered_athletes["Status"] == selected_status]

    filtered_athletes = filtered_athletes.groupby("ID").first().reset_index()
    filtered_athletes = filtered_athletes.sort_values(by=["First Name", "Last Name"])

    return [
        {"label": f"{row['First Name']} {row['Last Name']}", "value": row["ID"]}
        for _, row in filtered_athletes.iterrows()
    ]


# Callback to update athlete info and tab content
@app.callback(
    [Output("athlete-info", "children"), Output("tabs-content", "children")],
    [Input("athlete-dropdown", "value"), Input("tabs", "value")]
)
def update_dashboard(selected_id, selected_tab):
    if selected_id is None:
        return "Select an athlete to view details.", "Select a tab to view data."

    # Get athlete details
    athlete_tests = data[data["ID"] == selected_id].copy()
    athlete_tests["Date"] = pd.to_datetime(athlete_tests["Date"], errors="coerce")
    athlete_tests_sorted = athlete_tests.sort_values(by="Date", ascending=False)

    most_recent_level = athlete_tests_sorted["Level"].dropna().iloc[0] if not athlete_tests_sorted["Level"].dropna().empty else "Unknown"
    most_recent_school = athlete_tests_sorted["School/Org"].dropna().iloc[0] if not athlete_tests_sorted["School/Org"].dropna().empty else "Unknown"

    last_test_dates = {}
    today = pd.Timestamp.today()
    for workout in ["Strength", "Pitching", "Hitting"]:
        workout_tests = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == workout]
        if not workout_tests.empty:
            last_test_date = workout_tests.iloc[0]["Date"]
            days_since_test = (today - last_test_date).days
            last_test_dates[workout] = f"{last_test_date.strftime('%Y-%m-%d')} 🚨 Due for Testing" if days_since_test > 60 else last_test_date.strftime('%Y-%m-%d')
        else:
            last_test_dates[workout] = "No Test Recorded"

    athlete_info = athlete_tests_sorted.iloc[0]
    
    athlete_info_display = html.Div([
        html.H3(f"{athlete_info['First Name']} {athlete_info['Last Name']}"),
        html.P(f"ID: {selected_id}"),
        html.P(f"Status: {athlete_info['Status']}"),
        html.P(f"Trainer: {athlete_info['Trainer']}"),
        html.P(f"Position: {athlete_info['Position']}"),
        html.P(f"Level: {most_recent_level}"),
        html.P(f"School/Org: {most_recent_school}"),
        html.H4("Last Test Dates:"),
        html.P(f"Strength: {last_test_dates['Strength']}"),
        html.P(f"Pitching: {last_test_dates['Pitching']}"),
        html.P(f"Hitting: {last_test_dates['Hitting']}"),
    ])

    # Handle Tabs (Velocity Trends, Strength Trends, Test Comparisons)
    if selected_tab == "velocity":
        pitching_data = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == "Pitching"]

        if pitching_data.empty:
            tab_content = html.Div([html.H3("No velocity data available for this athlete.")])
        else:
            test_types = pitching_data["Test Type"].unique()
            graphs = []

            for test_type in test_types:
                test_data = pitching_data[pitching_data["Test Type"] == test_type]
                if test_data.empty:
                    continue

                y_axis = "Arm Score" if test_type == "ArmCare" else "Max Velo"
                y_label = "Arm Score" if test_type == "ArmCare" else "Max Velocity (MPH)"

            # 🔥 Find PRs for each Test Sub-Type
                pr_data = test_data.groupby("Test Sub-Type")[y_axis].transform(max)
                test_data["PR"] = test_data[y_axis] == pr_data  # Mark PR points for each Test Sub-Type

                fig = px.line(
                    test_data, x="Date", y=y_axis, color="Test Sub-Type", markers=True,
                    title=f"{test_type} - {y_label} Trends"
                )

            # 🔥 Add PR Annotations for each Test Sub-Type
                for _, row in test_data[test_data["PR"]].iterrows():
                    fig.add_annotation(
                        x=row["Date"], y=row[y_axis],
                        text=f"🔥 PR ({row['Test Sub-Type']})!", showarrow=True, arrowhead=2,
                        font=dict(color="red", size=12)
                    )

                fig.update_layout(xaxis_title="Date", yaxis_title=y_label)
                graphs.append(dcc.Graph(figure=fig))

            tab_content = html.Div(graphs)


    elif selected_tab == "strength":
        strength_data = athlete_tests_sorted[athlete_tests_sorted["Workout Type"] == "Strength"]

        if strength_data.empty:
            tab_content = html.Div([html.H3("No strength data available for this athlete.")])
        else:
            strength_tests = ["Back Squat", "Bench Press", "Deadlift"]
            graphs = []

            for test_type in strength_tests:
                test_data = strength_data[strength_data["Test Type"] == test_type]
                if test_data.empty:
                    continue

            # 🔥 Ensure date is formatted correctly (YYYY-MM-DD only)
                test_data["Date"] = test_data["Date"].dt.strftime('%Y-%m-%d')

            # 🔥 PR Detection: Find the highest Weight for each Test Sub-Type
                pr_weights = test_data.groupby("Test Sub-Type")["Weight"].max().reset_index()

            # 🔥 Within those, find the highest Speed for tiebreaker
                pr_data = pd.merge(test_data, pr_weights, on=["Test Sub-Type", "Weight"], how="inner")
                pr_data = pr_data.loc[pr_data.groupby("Test Sub-Type")["Speed"].idxmax()]

            # 🔥 Mark PR in the original dataset
                test_data["PR"] = test_data.apply(
                    lambda row: row["Weight"] == pr_data.loc[pr_data["Test Sub-Type"] == row["Test Sub-Type"], "Weight"].values[0]
                                and row["Speed"] == pr_data.loc[pr_data["Test Sub-Type"] == row["Test Sub-Type"], "Speed"].values[0]
                                if row["Test Sub-Type"] in pr_data["Test Sub-Type"].values else False,
                    axis=1
                )

            # 🔥 Create scatter plot (Weight vs. Speed)
                fig = px.scatter(
                    test_data, 
                    x="Speed", 
                    y="Weight", 
                    color="Test Sub-Type", 
                    title=f"{test_type} - Strength Trends",
                    hover_data={"Date": True, "Phase": True, "Weight": True, "Speed": True}  # Display extra info on hover
                )

            # 🔥 Add PR Annotations for each Test Sub-Type
                for _, row in test_data[test_data["PR"]].iterrows():
                    fig.add_annotation(
                        x=row["Speed"], y=row["Weight"],
                        text=f"🔥 PR ({row['Test Sub-Type']})!", showarrow=True, arrowhead=2,
                        font=dict(color="red", size=12)
                    )

                fig.update_layout(
                    xaxis_title="Speed (m/s)", 
                    yaxis_title="Weight (lbs)"
                )
                graphs.append(dcc.Graph(figure=fig))

            tab_content = html.Div(graphs)

    elif selected_tab == "test-comparisons":
        # Test Comparison Graphs
        comparison_data = athlete_tests_sorted[["Date", "Workout Type", "Max Velo", "Max Spin", "IVB", "Max Bat Speed", "Weight", "Speed"]].dropna(how="all")
        if comparison_data.empty:
            tab_content = html.Div([html.H3("No test comparison data available for this athlete.")])
        else:
            graphs = []
            pitching_data = comparison_data[comparison_data["Workout Type"] == "Pitching"]
            if not pitching_data.empty:
                fig_velo_spin = px.scatter(pitching_data, x="Max Velo", y="Max Spin", color="Workout Type",
                                           title="Max Velocity vs. Max Spin Rate")
                fig_velo_spin.update_layout(xaxis_title="Max Velocity (MPH)", yaxis_title="Max Spin Rate (RPM)")
                graphs.append(dcc.Graph(figure=fig_velo_spin))
            tab_content = html.Div(graphs)

    return athlete_info_display, tab_content

# Run the app in Jupyter Notebook
app.run_server(mode="inline", debug=True)
